In [39]:
using LinearAlgebra
# SVD algorithm
# Takes matrix A as input, outputs unitary matrices U, V and diagonal matrix S
# Such that A ≈ U*S*V^T

# Truncated SVD, computes only the t column vectors of U, t singular values, and t row vectors of V
# where t is an arbitrary positive integer
function truncatedSVD(A, t)
    m, n = size(A)
        
    if (n > m)
        throw(ArgumentError(A, "Matrix must be tall."))
    end

    # compute singular values
    sigma, singVec_V = eigen(hessenberg(Symmetric(A' * A)).H, n-t+1:n)
    print("\nsigma: ")
    show(stdout, "text/plain", sigma)
    
    
    # calculate r
    sig_rank = size(filter((x -> x != 0), sigma), 1)
    
    # does not work on complex eigen values
    # Calculate the squareroot of the eigenvalues of the A'A matrix, which are the eigenvalues of A
    sigma = broadcast((x -> sqrt(abs(x))), sigma)
    # Sort singVec_V and sigma so the eigenvalues are in descending order
    singVec_V = singVec_V[:, sortperm(sigma)]
    sigma = sort(sigma)

    # Compute left singular vectors
    # initializing the left singular vector
    singVec_U = Array{Float64}(undef, m, sig_rank)
        
    # Compute the left singular vector
    for i in 1:sig_rank
        # compute AV_i
        temp_A = A * singVec_V[:, i]
        # normalize AV_i
        temp_A_normal = temp_A / sigma[i]
        singVec_U[:,i] = temp_A_normal'
    end
    return singVec_U, sigma, singVec_V
end


truncatedSVD (generic function with 1 method)

In [68]:
A = rand(10,10)
myF = truncatedSVD(A, 5);
myU, myS, myV = myF;
F = svd(A)
U, S, V = F
A ≈ myU * Diagonal(myS) * myV'
print("\nA: ")
show(stdout, "text/plain", A)
print("\napproxA: ")
show(stdout, "text/plain", myU * Diagonal(myS) * myV')


sigma: 5-element Array{Float64,1}:
  0.6071687835514645
  0.7631292222952233
  1.8541055092054028
  2.630052860404346
 34.14142256387672
A: 10×10 Array{Float64,2}:
 0.998047  0.900547   0.0573522  0.928223  0.995109  0.573614   0.902437  0.683653   0.480113  0.580281
 0.537694  0.681054   0.677134   0.468123  0.829934  0.446747   0.870962  0.465126   0.109095  0.433063
 0.548296  0.155468   0.755607   0.205165  0.856211  0.56104    0.690952  0.455781   0.390484  0.618505
 0.727107  0.0655162  0.770151   0.665277  0.52892   0.799571   0.153714  0.259005   0.270657  0.812001
 0.588617  0.81033    0.991104   0.880083  0.240216  0.407828   0.826725  0.639832   0.428764  0.881526
 0.62872   0.945522   0.700122   0.432846  0.474775  0.0974964  0.643422  0.724474   0.10247   0.0518245
 0.225126  0.41879    0.944502   0.122853  0.223036  0.945625   0.564726  0.580062   0.971154  0.62276
 0.917303  0.475157   0.993203   0.70194   0.646198  0.727132   0.774231  0.732448   0.185698  0.551233
 0.

In [69]:
norm(A - U * Diagonal(S) * V') / norm(A)

1.463472611106965e-15